In [1]:
pip install BeautifulSoup


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "c:\Users\used\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\used\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\used\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\used\AppData\Local\Temp\pip-build-env-51zmi8ax\over

In [9]:
import requests
from bs4 import BeautifulSoup
import os
import urllib.parse
url  = "https://www.dhm.de/datenbank/ccp/prj_dhm_ccp/displayimg.php?laufnrid=cp146161_0&prj_short=dhm_ccp&format=gr&folder=badv"
# url = "https://www.dhm.de/datenbank/ccp/dhm_ccp.php?seite=6&fld_1=&fld_3=&fld_4=Objektfotos+%28BADV%29&fld_5=Abbildung&fld_6=&fld_6a=&fld_7=&fld_8=&fld_9=&fld_10=&fld_11=&fld_12_a=&fld_12_b=&fld_12a=&fld_13=&suchen=Search"
base_url = "https://www.dhm.de/datenbank/ccp/"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Create a directory to save images
os.makedirs('images', exist_ok=True)

# Find all image tags
img_tags = soup.find_all('img')

# Filter valid image URLs
def is_valid_image_url(url):
    parsed_url = urllib.parse.urlparse(url)
    return parsed_url.scheme in ['http', 'https'] and parsed_url.path.endswith(('.jpg', '.jpeg', '.png', '.gif'))
print(img_tags)
# Download and save each image
for img in img_tags:
    print(img)
    img_url = urllib.parse.urljoin(base_url, img['src'])
    
    if is_valid_image_url(img_url):
        img_name = os.path.join('images', os.path.basename(img_url))

        img_response = requests.get(img_url)
        with open(img_name, 'wb') as file:
            file.write(img_response.content)

        print(f"Downloaded {img_name}")

print("All images have been downloaded.")


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[]
All images have been downloaded.


In [13]:
import requests
from PIL import Image, ImageChops
from io import BytesIO
import os

base_url = "https://www.dhm.de/datenbank/ccp/prj_dhm_ccp/displayimg.php?laufnrid=cp{}_0&prj_short=dhm_ccp&format=gr&folder=badv"
start_num = 146161
end_num = 146170
# no_image_placeholder = 'images/cp146161_0.jpg'
no_image_placeholder = None

# Create a directory to save images
os.makedirs('images', exist_ok=True)

# Download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Check if two images are similar
def images_are_similar(img1, img2):
    return ImageChops.difference(img1, img2).getbbox() is None

# Get and save images
for i in range(start_num, end_num + 1):
    url = base_url.format(i)
    try:
        img = download_image(url)
        
        # Initialize the no_image_placeholder with the first "no image" placeholder found
        if no_image_placeholder is None and "no_image" in url.lower():
            no_image_placeholder = img
        
        if no_image_placeholder and images_are_similar(img, no_image_placeholder):
            print(f"Skipping placeholder image at {url}")
            continue

        img_name = os.path.join('images', f'cp{i}_0.jpg')
        img.save(img_name)
        print(f"Downloaded {img_name}")

    except Exception as e:
        print(f"Failed to download image from {url}: {e}")

print("All images have been processed.")


Downloaded images\cp146161_0.jpg
Downloaded images\cp146162_0.jpg
Downloaded images\cp146163_0.jpg
Downloaded images\cp146164_0.jpg
Downloaded images\cp146165_0.jpg
Downloaded images\cp146166_0.jpg
Downloaded images\cp146167_0.jpg
Downloaded images\cp146168_0.jpg
Downloaded images\cp146169_0.jpg
Downloaded images\cp146170_0.jpg
All images have been processed.


In [16]:
url = "https://www.dhm.de/datenbank/ccp/prj_dhm_ccp/displayimg.php?laufnrid=cp146161_0&prj_short=dhm_ccp&format=gr&folder=badv"

# def download_image(url):
    # response = requests.get(url)
    # img = Image.open(BytesIO(response.content))
    # return img
def download_image(url):
    response = requests.get(url)
    if response.headers['Content-Type'].startswith('image'):
        img = Image.open(BytesIO(response.content))
        return img
    else:
        return None

test_image = os.path.join('images', f'test.jpg')
img.save(test_image)
